In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [0]:
sources_name = "dev_catalog.bronze.shop_name_raw"
keys = ["shop_id"]
schema = {
    "shop_id": "int",
    "shop_name": "string",
    "branch_name": "string",
    "file_dt":"date"}

target_table = "dev_catalog.silver.shop_nameb_dwh"

In [0]:
# hard code
stage_1 = (
    spark.table("dev_catalog.bronze.shop_name_raw")
 # add sk column : key unique   
    .withColumn("sk",monotonically_increasing_id())
#   drop useless column
    .drop("_load_dt","_load_dttm")
           )
stage_1.display()

In [0]:

stage_1 = (
    spark.table(sources_name)
 # add sk column : key unique   
    .withColumn("sk",monotonically_increasing_id())
#   drop useless column
    .drop("_load_dt","_load_dttm")
           )
stage_1.display()

In [0]:
# function control column
def list_all_control_col(df:DataFrame)->list[str]:
    return[_column_name for _column_name in df.columns if _column_name.startswith("_")]

In [0]:
# hardcode

# stage 2 clear key is missing
# invalid datatype

int_rules = "^[0-9]+$"
date_rules = "^\\d{4}-\\d{2}-\\d{2}$"
invalid_df = (
    stage_1
    .withColumn("_shop_id_missing", col("shop_id").isNull())
    .withColumn("_shop_id_invalid",when(col("shop_id").rlike(int_rules),False).otherwise(True))
    .withColumn("_file_dt_invalid",when(col("file_dt").rlike(date_rules),False).otherwise(True))
              )

invalid_df.display()

In [0]:
# stage 2 clear key is missing
# invalid datatype

data_rules = {
    "int": "^[0-9]+$",
    "date" : "^\\d{4}-\\d{2}-\\d{2}$"
}

# key is null
for _col_name in keys:
    invalid_df = stage_1.withColumn(f"_{_col_name}_missing", col(_col_name).isNull())

# invalid datatype
for _col_name,_data_type in schema.items():
    if _data_type == "string":
        continue
    invalid_df = invalid_df.withColumn(f"_{_col_name}_invalid",when(col(_col_name).rlike(data_rules[_data_type]),False).otherwise(True))
    
invalid_col = " OR ".join(list_all_control_col(invalid_df))

invalid_df = invalid_df.filter(invalid_col)

invalid_df.display()

In [0]:
stage_2 = (
    stage_1
    .join(invalid_df,["sk"], "left_anti")
)

stage_2.display()

In [0]:
list(schema.keys())

In [0]:
# hard code
# row duplicate
partition_by_all = Window.partitionBy("shop_id","shop_name","branch_name","file_dt").orderBy("shop_id")

row_dup_df = (
  stage_2
  .withColumn("rn", row_number().over(partition_by_all))
  .filter(col("rn")>1)
  .drop("rn")
  .withColumn("_remarks",lit("row_dup"))
)
row_dup_df.display()

partition_by_key = Window.partitionBy("shop_id").orderBy("shop_id")

key_dup_df = (
    stage_2
    .join(row_dup_df, ["sk"],"left_anti")
    .withColumn("rn", count("*").over(partition_by_key))
    .filter(col("rn") == 2)
    .drop("rn")
    .withColumn("_remarks",lit("key_dup"))
)
key_dup_df.display()

In [0]:

# row duplicate
partition_by_all = Window.partitionBy(list(schema.keys())).orderBy(keys)

row_dup_df = (
  stage_2
  .withColumn("rn", row_number().over(partition_by_all))
  .filter(col("rn")>1)
  .drop("rn")
  .withColumn("_remarks",lit("row_dup"))
)
row_dup_df.display()

partition_by_key = Window.partitionBy(*keys).orderBy(keys)

key_dup_df = (
    stage_2
    .join(row_dup_df, ["sk"],"left_anti")
    .withColumn("rn", count("*").over(partition_by_key))
    .filter(col("rn") == 2)
    .drop("rn")
    .withColumn("_remarks",lit("key_dup"))
)
key_dup_df.display()

# combine key_dup + row_dup
duplicate_df = row_dup_df.union(key_dup_df)
duplicate_df.display()

In [0]:
stage_3 = (
    stage_2
    .join(duplicate_df,["sk"],"left_anti")
)

stage_3.display()

In [0]:
# hard code
cast_df = (
    stage_3
    .select(
        col("sk")
        ,col("shop_id").cast("int")
        ,col("shop_name").cast("string")
        ,col("branch_name").cast("string")
        ,col("file_dt").cast("date")
        
    )
)

cast_df.display()

In [0]:
select_cast_col = [col(_col_name).cast(_data_type) for _col_name,_data_type in schema.items()]
print(*select_cast_col)

In [0]:

cast_df = (
    stage_3
    .select(
        *select_cast_col
        ,col("sk")
    )
)
      

cast_df.display()

filter_error should be run manual so cannot make in dynamic system

In [0]:
# error_df = (
#     cast_df
#     .filter(col("branch_name").isNull())
# )
# print("error_df")
# error_df.display()

final_df = (
    cast_df
    # .join(error_df,["sk"],"left_anti")
    .withColumn("_insert_date",current_date())
    .withColumn("_update_date",current_date())
    .withColumn("_last_mod_ts",current_timestamp())
    .drop("sk")
    .withColumn("shop_name",lit("new_shop_name"))
)
print("final_df")
final_df.display()


In [0]:
pipeline_mode = "overwrite"

In [0]:

# spark.sql("drop table if exists dev_catalog.silver.shop_name_dwh")

final_df.write.mode(pipeline_mode).saveAsTable(
    target_table
)

# spark.table(target_table).display()

In [0]:
spark.table("dev_catalog.silver.shop_nameb_dwh").display()